# Tutorial 04 - Results Analysis
Having performed some learning processes, this tutorial explains how to apply available tools in the CommonRoad-RL package to demonstrate results. These include
* how to plot learning curves of rewards and other information
* how to render a trained model in various scenarios over entire episodes
* how to generate a CommonRoad solution for system submission

## 0. Preparation

Please check the followings:
* current path is at the project root `commonroad-rl`, i.e. two upper layers to the `tutorials` folder
* interactive python kernel is triggered from the correct environment

In addition, please make sure the following files exist:  
* a processed data directory at `tutorials/data/highD/pickles`
* a trained model at `tutorials/logs/best_model.zip`
* a saved environment wrapper `tutorials/logs/vecnormalize.pkl`
* a monitored information file `tutorials/logs/infos.monitor.csv`
* a recorded evaluation file `tutorials/logs/evaluations.npz`

Otherwise, see **Tutorial 01 - Data Preprocessing**, **Tutorial 02 - Vanilla Learning** and **Tutorial 03 - Continual Learning**.

In [ ]:
# Check current path
%cd ../..
%pwd

# Check interactive python kernel
import sys
sys.executable

## 1. Plot learning curves
A learning process is usually recorded with information such as reward values and terminating conditions, resulting in a `.monitor.csv` file. In out case, we have `tutorials/logs/infos.monitor.csv` and `tutorials/logs/test/infos.monitor.csv`.
 
After a learning process is finished, we need to check the convergence of the model. Simply use `commonroad_rl/plot_learning_curves.py` to plot out the learning curves from the recorded data. 

In [ ]:
!python -m commonroad_rl.plot_learning_curves \
          --log-folder commonroad_rl/tutorials \
          --model_path logs \
          --legend_name ppo2 #\
          # --smooth

## 2. Render the best model

To see the learned behavior of the ego vehicle in a scenario, we apply `commonroad_rl/evaluate_model.py`, which in turn employs the rendering function defined in `commonroad_rl/gym_commonroad/commonroad_env.py`. As an example, we do this by calling this file with a terminal command, using our result from **Tutorial 03 - Continual Learning**.

In [ ]:
# Call python script to render a trained model in a scenario
!python -m commonroad_rl.evaluate_model \
          --algo ppo2 \
          --model_path commonroad_rl/tutorials/logs \
          --test_path commonroad_rl/tutorials/data/highD/pickles \
          --viz_path commonroad_rl/tutorials/imgs

As seen from the output messages, the model is verified by various scenarios in the `tutorials/data/highD/pickles` directory and there should be rendered images under `tutorials/imgs` now.

## 3. Perform Sensitivity Analysis
To evaluate the importance of a specific observation, we apply `commonroad_rl/sensitivity_analysis.py` on the trained model, which uses the [SALib library](https://salib.readthedocs.io/en/latest/) .
The sensitivity analysis can apply 5 different algorithms. The algorithms and their respective output with interpretation are described below:
- Sobol Sensitivity Analysis (default/ `--method sobol`)
    - S1 (first‐order sensitivity indices as the main effect are used to measure the fractional contribution of a single parameter to the output variance - The higer the value, the more critical is the input parameter for the output)
    - S2 (Second‐order sensitivity indices are used to measure the fractional contribution of parameter interactions to the output variance - correlation of two inputs weighted by the effect they have on the output)
    - ST (Total‐order sensitivity indices take into account both the main, second‐order and higher‐order effects)
    - S1_conf/ S2_conf/ ST_conf (Confidence intervals of the individual sensitivity indices)

- FAST - Fourier Amplitude Sensitivity Test (`--method fast`)
    - S1 (first-order sensitivity)
    - ST (total-order sensitivity)

- RBD-FAST - Random Balance Designs Fourier Amplitude Sensitivity Test (`--method rbd_fast`)
    - S1 (first-order sensitivity)

- Method of Morris (`--method morris`)
    - mu (one-step-at-a-time first order sensitivity (onyl one input is changed from one input to the next)) (Can contain errors when the function is non-monotonic. If mu is low and sigma for the respective observation is high, the mu shows a wrong value --> mu_star resolves this issue)
    - mu_star (one-step-at-a-time first order sensitivity with absolute values)
    - sigma (standard deviation of the sensitivity of an observation)
    - sigma_conf (sigma confidence)

    
- Delta Moment-Independent Measure (`--method delta`)
    - delta (distance measure between the output probability and the output probability conditioned on the respecitve variable)
    - delta_conf
    - S1 (first-order sensitivity based on sobol)
    - S1_conf

Note that the results are saved in `tutorials/logs/sens_analysis_figs`

In [ ]:
!python -m commonroad_rl.sensitivity_analysis \
    --algo ppo2 \
    --method sobol \
    --model_path commonroad_rl/tutorials/logs \
    --save_path commonroad_rl/tutorials/logs \
    --data_path commonroad_rl/tutorials/data/highD/pickles \
    --save_fig

## 4. Generate a CommonRoad solution
Similarly, `commonroad_rl/generate_solution.py` helps generate CommonRoad solutions from successful trained models. For example, we run the following command.
It might be that you have to adapt the test path to the location of your test pickle-files. In addition, the meta_scenario folder has to be located at:  `tutorials/data/highD/meta_scenario`

In [ ]:
!python -m commonroad_rl.generate_solution \
          --algo ppo2 \
          --model_path commonroad_rl/tutorials/logs \
          --test_path commonroad_rl/tutorials/data/highD/pickles \
          --solution_path commonroad_rl/tutorials/sols

Again the model is run with the testing scenarios, yet there are no actual solutions being generated under `tutorials/sols` because the goal is not reached in any cases. If a powerful model is obtained, calling the Python file shall generate CommonRoad solutions for all goal-reaching scenarios. Then you can upload your solutions to the [CommonRoad website](https://commonroad.in.tum.de/) to compete with motion planner researchers all over the world!